# Trabajo Práctico 1 - Matrices de Insumo Producto

## Enunciado

### Consigna 1 - 

Vamos a interpretar las posibles soluciones que podemos obtener del sistema $$ p = (I - A)^{-1} d = Ld $$ Para ello tendremos en cuenta que significa que una matriz sea inversible o no y como impacta esto en las soluciones de mi sistema de ecuaciones lineales.
<p style="text-align: left;">
a) \( (I - A) \ \text{es inversible} \)
</p>
Si la matriz (I - A) tiene inversa esto significa que existe la matriz de Leontief, luego existe solucion unica para este sistema.
<p></p>
<p style="text-align: left;">
b) \( (I - A) \ \text{es inversible y d = 0} \)
</p>
En este caso existe matriz de Leontief pero mi sistema resulta en $p = (I - A)^{-1}\cdot0 = L\cdot0 = 0$  Podemos decir que no existe demanda externa en este caso y que la produccion total es destinada a satisfacer la demanda interna.
<p></p>
<p style="text-align: left;">
c) \( (I - A) \ \text{no es inversible} \)
</p>
Al no existir inversa esto significa que no se puede calcular la matriz de Leontief. Esto tiene dos posibles conclusiones:  <p></p> 
  . El sistema no tiene solucion. <p></p> 
  . Existen infinitas soluciones debido que hay dependencia lineal entre las ecuaciones de mi sistema. Esto puede ocurrir debido a una relacion entre sectores que determinan que no hay unica solucion.
<p style="text-align: left;">
d) \( (I - A) \ \text{no es inversible y d = 0} \)
</p>    
Debido a que no existe matriz de Leontief y que d = 0, para este caso el sistema admite infinitas soluciones para p.



### Consigna 2 -  

In [2]:
import numpy as np
from scipy.linalg import solve_triangular

def calcularLU(A):
    """
    Calcular la factorización LU de una matriz.

    Args:
        A (numpy.ndarray): Matriz cuadrada que se desea factorizar.

    Returns:
        L (numpy.ndarray): Matriz triangular inferior L.
        U (numpy.ndarray): Matriz triangular superior U.
        P (numpy.ndarray): Matriz de permutación P.
    """
    m = A.shape[0]
    n = A.shape[1]
    Ac = A.copy()
    P = np.eye(n)
    
    if m != n:
        print('Matriz no cuadrada')
        return

    L = np.eye(n) 
        
    for i in range(0, n):   
        max_row = np.argmax(np.abs(Ac[i:n, i])) + i
        if i != max_row:
            # Intercambiamos las filas en Ac
            Ac[[i, max_row], :] = Ac[[max_row, i], :]
            
            # Intercambiamos las filas en P
            P[[i, max_row], :] = P[[max_row, i], :]
            
            # Intercambiamos las filas en L hasta la columna i (excluyendo la diagonal)
            if i > 0:
                L[[i, max_row], :i] = L[[max_row, i], :i]
            
        for j in range(i+1, n):
            piv = Ac[j][i] / Ac[i][i]
            Ac[j] = Ac[j] - piv * Ac[i]
            L[j][i] = piv

    U = Ac
    return L, U, P

---

In [3]:

def inversaLU(L, U, P):
    """
    Calculo de la inversa de una matriz a partir de su factorización LU.

    Args:
        L (numpy.ndarray): Matriz triangular inferior L.
        U (numpy.ndarray): Matriz triangular superior U.
        P (numpy.ndarray): Matriz de permutación P.

    Returns:
        numpy.ndarray: Matriz inversa de la matriz original A que fue factorizada en L, U y P.
    """
    n = L.shape[0]
    Inv = np.zeros((n, 0)) 

    if P is None:
        P = np.eye(n)  # Si no se proporciona P, se usa la identidad
    for i in range(n):
        b = P[:, i]  
        x = solve_triangular(L, b, lower=True)
        y = solve_triangular(U, x, lower=False)
        Inv = np.column_stack((Inv, y))  # Agregamos la columna de la inversa

    return Inv


### Consigna 3 -  

$$
A = \begin{pmatrix}
0.3 & 0.0 & 0.1 \\
0.05 & 1.0 & 0.2 \\
0.1 & 0.15 & 0.1
\end{pmatrix}
, \quad
d = \begin{pmatrix}
100 \\
100 \\
300
\end{pmatrix}
$$


In [12]:
A = np.array([[0.3,0.0,0.1],[0.05,1.0,0.2],[0.1,0.15,0.1]])
d = np.array([100,100,300])

L,U,P = calcularLU(np.eye(3) - A)

print("P = " + str(inversaLU(L, U, P)@d))


P = [   68.96551724 -5149.42528736  -517.24137931]


a)
En el contexto de un modelo de insumo-producto como el de Leontief, el vector de producción $ P $
indica la cantidad de bienes o servicios que deben producirse en cada sector para satisfacer la demanda interna y externa.
En nuestro caso podemos explicar los requerimientos de cada sector de $ P $ de la siguiente forma :

Primer sector ($ 68.97 $): El primer valor del vector indica que el primer sector debe producir 68.97 unidades. Esta cantidad satisface tanto la demanda externa como los insumos que requieren los otros sectores. Por lo tanto este sector se encuentra en superávit.

Segundo sector ($ -5149.43 $): Un valor negativo sugiere que la producción en el segundo sector no solo es insuficiente para cubrir la demanda externa, sino que probablemente es un insumo significativo para otros sectores. Este déficit indica que se requiere una gran cantidad de productos del segundo sector como insumos para producir en otros sectores, lo que crea una dependencia significativa. Por lo tanto este sector se encuentra en déficit

Tercer sector ($ -517.24 $): Similar al segundo sector, este valor negativo implica un consumo neto de insumos del tercer sector. El sistema consume 517.24 unidades del tercer sector para satisfacer las demandas de otros sectores o la demanda final. Por lo tanto este sector se encuentra en déficit.

b)
La caracteristica de la matriz de Leontief $ L = (I-A) $ que lleva a esta solucion unica, es que es inversible. Es decir, su determinante es distinto de cero, por lo que la economía descrita por la matriz A es autosuficiente y sostenible en su estructura. Esto implica que cada sector, aunque dependa de otros, es capaz de satisfacer la demanda externa y su demanda interna de insumos. Si el determinante fuera cero, indicaría que hay una dependencia excesiva o una redundancia en el sistema. Por ejemplo, un sector podría necesitar infinitas unidades de otro sector para producir, lo cual es inviable.


### Consigna 4 -  

Ante un shock de demanda externa $ \Delta d = \begin{pmatrix} 0 \\ 0 \\ 1 \end{pmatrix} $ , la producción total se tendrá que adaptar bajo la ecuación

$$ p' = L \cdot d' = p + \Delta p $$

siendo $ \Delta p $ el cambio de producción tal que  $ \Delta p = L \cdot \Delta d $

Teniendo en cuenta que $ \Delta d $ solo tiene un valor en la tercer columna, el shock de demanda se concentra en el tercer producto, pero genera cambios en la producción de todos los demás.

Analíticamente, siendo la matriz $ L $ la matriz de Leontief:

$$
L = \begin{pmatrix}
L_{11} & L_{12} & L_{13} \\
L_{21} & L_{22} & L_{23} \\
L_{31} & L_{32} & L_{33}
\end{pmatrix}
$$

El cambio en la producción se calcula como:

$$ \Delta p = L \cdot \begin{pmatrix} 0 \\ 0 \\ 1 \end{pmatrix} = \begin{pmatrix} L_{13} \\ L_{23} \\ L_{33} \end{pmatrix} $$

Esto coincide con la observación anterior: un shock de demanda en el tercer producto afecta a la producción de todos los demás.

### Consigna 5 -  

In [ ]:
## Sea Z mi matriz compuesta por la demanda interna de cada sector
Z = np.array([[350,0,0],[50, 250, 150], [200,150,550]])
## vector p que muestra la produccion total por cada sector
p = np.array([1000, 500, 1000])
## Armamo mi matriz diagonal P
ProdTotal = np.diag(p)
## Calculamos la matriz A con coeficientes tecnicos, A = Z@P^-1, utilizando las funciones calcularLU e inversaLU
L, U, P = calcularLU(ProdTotal)
pInv = (inversaLU(L,U,P))
A = Z@pInv
print(A) 
## Calculamos la matriz de Leontief: L=(I - A)^-1 
## (I - A)
I = np.eye(Z.shape[0])
Lp = I - A
## Matriz de Leontief
L, U, P = calcularLU(Lp)
matriz_Leontief = (inversaLU(L,U,P))
print(matriz_Leontief)

### Consigna 6 -  

A partir de la fórmula
$$

\begin{pmatrix}

\begin{pmatrix}
I_{nxn} & 0_{nxm} \\
0_{mxn} & I_{mxm} \\ 
\end{pmatrix}
-
\begin{pmatrix}
A^{rr} & A^{rs} \\
A^{sr} & A^{ss} \\ 
\end{pmatrix}

\end{pmatrix}

\begin{pmatrix} p^r \\ p^s \end{pmatrix}

=

\begin{pmatrix} d^r \\ d^s \end{pmatrix}

$$
Queremos encontrar $ \Delta p^r$


Para despejar, restamos a las Identidades $ I $ las matrices $ A $ y luego, aplicamos distributiva para los vectores de producción $ p $

$$

\begin{pmatrix}
(I_{nxn} - A^{rr}) p^r - A^{rs} p^s \\
- A^{sr} p^r + (I_{mxm} - A^{ss}) p^s\\ 
\end{pmatrix}

=

\begin{pmatrix} d^r \\ d^s \end{pmatrix}

$$


Separamos las 2 filas en 2 ecuaciones para formar el sistema

$$

\begin{pmatrix}
(I_{nxn} - A^{rr}) p^r - A^{rs} p^s
\end{pmatrix}

=

d^r 

$$
$$

\begin{pmatrix}
-A^{sr} p^r + (I_{mxm} - A^{ss}) p^s
\end{pmatrix}

=

d^s 

$$


Derivamos ambas partes de la igualdad para las 2 ecuaciones, quedando

$$
\begin{pmatrix}
(I_{nxn} - A^{rr}) \Delta p^r - A^{rs} \Delta p^s
\end{pmatrix}

=

\Delta d^r 

$$
$$

\begin{pmatrix}
-A^{sr} \Delta p^r + (I_{mxm} - A^{ss}) \Delta p^s
\end{pmatrix}

=


\Delta d^s 

$$



Tenemos como dato que $ \Delta d^s = 0 $, Esto implica que
$$
(I_{mxm} - A^{ss}) \Delta p^s = A^{sr} \Delta p^r
$$

$$
\Delta p^s = (I_{mxm} - A^{ss})^{-1} A^{sr} \Delta p^r 
$$


Reemplazamos en la otra ecuacion

$$

\begin{pmatrix}
(I_{nxn} - A^{rr}) \Delta p^r - A^{rs} 
(I_{mxm} - A^{ss})^{-1} A^{sr} \Delta p^r 
\end{pmatrix}

=

\Delta d^r 

$$


Y despejamos el resultado

$$ 
\Delta p^r ( (I_{nxn} - A^{rr}) - A^{rs} 
(I_{mxm} - A^{ss})^{-1} A^{sr} ) = \Delta d^r 
$$

$$
\Delta p^r = ( (I_{nxn} - A^{rr}) - A^{rs} (I_{mxm} - A^{ss}) ^ {-1} A^{sr} ) ^ {-1} \Delta d^r 
$$


Que, usando la asociatividad en la adición en matrices es igual al de la consigna

$$
\Delta p^r = ( I_{nxn} - A^{rr} - A^{rs} (I_{mxm} - A^{ss}) ^ {-1} A^{sr} ) ^ {-1} \Delta d^r 
$$

### Consigna 7 -  

In [ ]:
import pandas as pd
import numpy as np

###############  Codigo para el armado de dataframes ############################

#df = pd.read_csv('Downloads/matrizlatina2011_compressed_0.csv')  
#PER = df[df['Country_iso3'] == 'PER']    
#Nic = df[df['Country_iso3'] == 'NIC']
#PER = pd.read_csv('ALC/peru.csv')  
#NIC = pd.read_csv('ALC/Nicaragua.csv') 


In [ ]:

# Generar vectores de 'producción total' y submatrices Z
Pp = PER['Output'].values
Pn = NIC['Output'].values
Zpn = PER.filter(like='NIC').drop(["CONS_NIC", "GFCF_NIC"], axis=1).values
Zpp = PER.filter(like='PER').drop(["CONS_PER", "GFCF_PER"], axis=1).values
Znn = NIC.filter(like='NIC').drop(["CONS_NIC", "GFCF_NIC"], axis=1).values
Znp = NIC.filter(like='PER').drop(["CONS_PER", "GFCF_PER"], axis=1).values


In [ ]:
import matplotlib.pyplot as plt

# Función para evitar ceros en la producción
def ajustar(P):
    for i in range(len(P)):
        if P[i] == 0:
            P[i] = 1
    return P
# Ajustamos producciones
Pn = ajustar(Pn)
Pp = ajustar(Pp)
# Calculamos matrices inversas
PnInv = np.diag(1 / Pn)
PpInv = np.diag(1 / Pp)
# Construimos matriz A
App = Zpp @ PpInv
Apn = Zpn @ PnInv
Anp = Znp @ PpInv
Ann = Znn @ PnInv
A = np.block([[App, Apn], [Anp, Ann]])

#Construir matriz Leontief y aplicar shocks 
# Calcular demanda final para el pais P1 (Perú)(SIN SHOCK)
I = np.eye(A.shape[0])  # Matriz identidad
#Shock  negativo
delta_d = np.zeros(d.shape)
delta_d[5]= -d[5]*0.10 
#shock positivo
for i in range(6,9):
    porcentaje = 0.033*d[i]
    delta_d[i]+= porcentaje

d_prima = d + delta_d

#Produccion sin shocks
p = np.concatenate([Pp, Pn])
#Produccion despues del shock P= (I-A) * d_prima
L, U, P = calcularLU(I- A)
Leontief = (inversaLU(L,U,P))
p_prima = Leontief @ d_prima

delta_p = p_prima - p

########## Grafico comparandos la produccion con shock vs sin shock
prod_sin_shock =  p[:10] 
prod_con_shock =  p_prima[:10] 
indices = np.arange(1, 11)
plt.figure(figsize=(10, 6))
plt.plot(indices, prod_sin_shock, marker='o', label='Producción sin shocks (p)')
plt.plot(indices, prod_con_shock, marker='o', label='Producción después del shock (p_prima)')
plt.title('Comparación de Producción: p vs p_prima')
plt.xlabel('Sector')
plt.ylabel('Valor')
plt.xticks(indices)  
plt.legend()
plt.grid()
plt.show()


Como podemos observar en los primeros diez componentes del vector p y p_prima
que coinciden con la produccion de cada sector, no se aprecia un diferencia significativa luego de aplicar el shock
en la demanda. Sin embargo podriamos llegar a notar una leve diferencia en los sectores 6,7 y 8 que fue 
donde se aplico el shock


In [ ]:

# -Modelo de región simple-
# Peru "pp" es región R
# Cálculo Demanda
Ipp = np.eye(App.shape[0]) 
DppSinShock = (Ipp - App) @ Pp
DppConShock = (Ipp - App) @ Pp
DppConShock[0:40] += 1  # Demanda con shock arbitrario de 1.

# Recalculo Producción
Lpp, Upp, Ppp = calcularLU(Ipp - App)  # LU de Leontief Perú
LeontiefInvpp = inversaLU(Lpp, Upp, Ppp)
Producción = (LeontiefInvpp @ DppSinShock)  # ≈Pp
ProducciónConShock = (LeontiefInvpp @ DppConShock)
deltaPpp = ProducciónConShock - Producción

# -Fórmula completa-
# Peru "pp" sigue siendo región R & Nicaragua "nn" es región S

# Calculo efectos de Ann en App segun la fórmula
Inn = np.eye(Ann.shape[0])  
Lnn,Unn,Pnn = calcularLU(Inn-Ann)
LeontiefInvnn = inversaLU(Lnn,Unn,Pnn)
AppC = (Ipp-App-Apn@LeontiefInvnn@Anp) # Leontief Perú Completa. 

#Mismos calculos, notar que AppC incluye Ipp (en los cálculos anteriores, App no la incluia)
# Cálculo Demanda
DppSinShockC = (AppC) @ Pp
DppConShockC = (AppC) @ Pp
DppConShockC[0:40] += 1  # Demanda con shock arbitrario de 1.

# Recalculo Producción
Lpp, Upp, Ppp = calcularLU(AppC)  # LU de Leontief Perú Completa
LeontiefInvpp = inversaLU(Lpp, Upp, Ppp)
Producción = (LeontiefInvpp @ DppSinShockC)  # ≈Pp
ProducciónConShock = (LeontiefInvpp @ DppConShock)
deltaPppC = ProducciónConShock - Producción

# -Comparación de diferencias entre ΔP de formula simple & ΔPC de formula completa-

print(np.isclose(deltaPpp, deltaPppC))
DiferenciaEntreDeltas = deltaPppC - deltaPpp

# Gráfico comparativo
fig, axs = plt.subplots(1, 1, figsize=(12, 5))

# Graficar la diferencia (Delta P)
axs.plot(DiferenciaEntreDeltas, color='orange')
axs.set_title('Cambio en ΔP entre fórmula simple y completa')
axs.set_xlabel('Sectores')
axs.set_ylabel('Δ Producción')

plt.tight_layout()
plt.show()

Notamos que el cambio de Producción $ \Delta P $ por un shock de demanda $ \Delta d $ no sufre una diferencia significativa al ser medido por el modelo de región simple y el que considera las relaciones interregionales. Esto representa que la Matriz insumo-producto de Perú tiene poca dependencia de la de Nicaragua.